# 🍏 Fun & Fit Health Advisor Agent Tutorial 🍎

Welcome to our **Fun & Fit Health Advisor Agent** tutorial, where you'll use **Azure AI Foundry** SDKs to create a playful (yet carefully disclaimed!) health and fitness assistant. We'll:

1. **Initialize** our project using **azure-ai-projects**.
2. **Create an Agent** specialized in providing general wellness and nutritional advice (with disclaimers!).
3. **Manage conversations** about fitness, nutrition, and general health topics.
4. **Showcase logging and tracing** with **OpenTelemetry**.
5. **Demonstrate** how to incorporate tools, safety disclaimers, and basic best practices.

### ⚠️ Important Medical Disclaimer ⚠️
> **The health information provided by this notebook is for general educational and entertainment purposes only and is not intended as a substitute for professional medical advice, diagnosis, or treatment.** Always seek the advice of your physician or other qualified health provider with any questions you may have regarding a medical condition. Never disregard professional medical advice or delay seeking it because of something you read or receive from this notebook.




## 1. Initial Setup
We'll start by importing needed libraries, loading environment variables, and initializing an **AIProjectClient** so we can do all the agent-related actions. Let's do it! 🎉


In [4]:
# Import required libraries
import os  # For environment variables and path operations
import time  # For adding delays if needed
from pathlib import Path  # For cross-platform path handling
import json


# Import Azure and utility libraries
from azure.identity import DefaultAzureCredential  # For Azure authentication
from azure.ai.projects import AIProjectClient  # Main client for AI Projects

def find_cred_json(start_path):
    current = Path(start_path)
    while current != current.parent:
        cred_file = current / 'cred.json'
        if cred_file.exists():
            return str(cred_file)
        current = current.parent
    return None

start_dir = os.getcwd()  # Use current directory
file_path = find_cred_json(start_dir)

print(f"Found cred.json at: {file_path}")

try:
    with open(file_path, 'r') as f:
        loaded_config = json.load(f)

    print("Project Endpoint:", loaded_config['PROJECT_ENDPOINT'])
    print("Model Deployment ID:", loaded_config['MODEL_DEPLOYMENT_NAME'])

    project_client = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=loaded_config["PROJECT_ENDPOINT"],
    )

    print("✅ Successfully initialized AIProjectClient")

except FileNotFoundError:
    print(f"❌ Could not find file at: {file_path}")
except json.JSONDecodeError:
    print("❌ File exists but contains invalid JSON")
except TypeError:
    print("❌ File path was None — cred.json not found in any parent directories.")


Found cred.json at: /workspaces/Azure-ai-foundry01/cred.json
Project Endpoint: https://sarath-8734-resource.services.ai.azure.com/api/projects/sarath-8734
Model Deployment ID: gpt-4o
✅ Successfully initialized AIProjectClient


## 2. Creating our Fun & Fit Health Advisor Agent 🏋️

We'll create an Agent specialized in general health and wellness. We'll explicitly mention disclaimers in its instructions, so it never forgets to keep it safe! The instructions also ask the agent to focus on general fitness, dietary tips, and always encourage the user to seek professional advice.


In [5]:
def create_health_advisor_agent():
    """Create a health advisor agent with disclaimers and basic instructions."""
    try:
        # Get the model name from environment variables, default to gpt-4o-mini if not set
        model_name = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o")

        # Create a new agent using the AIProjectClient
        # The agent will use the specified model and follow the given instructions
        agent = project_client.agents.create_agent(
            model=model_name,
            name="fun-fit-health-advisor",
            # Define the agent's behavior and responsibilities
            instructions="""
            You are a friendly AI Health Advisor.
            You provide general health, fitness, and nutrition information, but always:
            1. Include medical disclaimers.
            2. Encourage the user to consult healthcare professionals.
            3. Provide general, non-diagnostic advice around wellness, diet, and fitness.
            4. Clearly remind them you're not a doctor.
            5. Encourage safe and balanced approaches to exercise and nutrition.
            """
        )
        # Log success and return the created agent
        print(f"🎉 Created health advisor agent, ID: {agent.id}")
        return agent
    except Exception as e:
        # Handle any errors during agent creation
        print(f"❌ Error creating agent: {str(e)}")
        return None

# Create an instance of our health advisor agent
health_advisor = create_health_advisor_agent()

🎉 Created health advisor agent, ID: asst_fLv7hapk1BuVkjNNJsjyD6nm


## 3. Managing Our Health Conversations 💬
A conversation (or *thread*) is where we'll store the user's messages and the agent's responses about health topics. Let's create a new thread dedicated to Health & Fitness Q&A.


In [9]:
def start_health_conversation():
    try:
        thread = project_client.agents.threads.create()
        print(f"📝 Created a new conversation thread, ID: {thread.id}")
        return thread
    except Exception as e:
        print(f"❌ Error creating thread: {e}")
        return None
    
health_thread = start_health_conversation()


📝 Created a new conversation thread, ID: thread_YEJct9rjEsJ43A9JLZAoGglT


## 4. Asking Health & Fitness Questions 🏃‍♂️
We'll create messages from the user about typical health questions. For example, **"How do I calculate my BMI?"** or **"What's a balanced meal for an active lifestyle?"**. We'll let our Health Advisor Agent respond, always remembering that disclaimer!


In [10]:
def ask_health_question(thread_id, user_question):
    try:
        return project_client.agents.messages.create(
            thread_id=thread_id,
            role="user",
            content=user_question
        )
    except Exception as e:
        print(f"❌ Error adding user message: {e}")
        return None

def process_thread_run(thread_id, agent_id):
    try:
        run = project_client.agents.runs.create(
            thread_id=thread_id,
            agent_id=agent_id
        )
        while run.status in ["queued", "in_progress", "requires_action"]:
            time.sleep(1)
            run = project_client.agents.runs.get(
                thread_id=thread_id,
                run_id=run.id
            )
        print(f"🤖 Run completed with status: {run.status}")
        return run
    except Exception as e:
        print(f"❌ Error processing thread run: {e}")
        return None

def view_thread_messages(thread_id):
    try:
        messages = project_client.agents.messages.list(thread_id=thread_id)
        print("\n🗣️ Conversation so far (oldest to newest):")
        for m in messages:
            print(f"{m.role.upper()}: {m.content}\n")
        print("-----------------------------------\n")
    except Exception as e:
        print(f"❌ Error viewing thread: {e}")

### Example Queries
Let's do some quick queries now to see the agent's disclaimers and how it handles typical health questions. We'll ask about **BMI** and about **balanced meal** for an active lifestyle.


In [11]:
if health_advisor and health_thread:
    ask_health_question(health_thread.id, "How do I calculate my BMI, and what does it mean?")
    process_thread_run(health_thread.id, health_advisor.id)
    ask_health_question(health_thread.id, "Can you give me a balanced meal plan for someone who exercises 3x a week?")
    process_thread_run(health_thread.id, health_advisor.id)
    view_thread_messages(health_thread.id)
else:
    print("❌ Setup incomplete – agent or thread is missing.")


🤖 Run completed with status: RunStatus.COMPLETED
🤖 Run completed with status: RunStatus.COMPLETED

🗣️ Conversation so far (oldest to newest):
ASSISTANT: [{'type': 'text', 'text': {'value': "Of course! A balanced meal plan helps fuel your body, keep energy levels stable, and support recovery for someone exercising three times a week. Below is a **general, healthy, and balanced meal plan**. This plan is customizable and intended for a person exercising moderately (e.g., strength training, cardio, or other 30-60 minute sessions). The exact portions will vary based on specific caloric needs, gender, activity level, and fitness goals (weight loss, maintenance, or gain).\n\n---\n\n### **General Guidelines for Balanced Meals**:\n1. **Macros**:\n   - **Carbohydrates (40-50%)**: Fuel your exercise and keep energy up.\n   - **Protein (20-30%)**: Supports muscle repair and recovery.\n   - **Fats (20-30%)**: For overall health and sustained energy.\n   \n2. **Hydration**:\n   - Drink plenty of wat

## 5. Cleanup 🧹
If you'd like to remove your agent from the service once finished, you can do so below. (In production, you might keep your agent around for stateful experiences!)

In [ ]:
def cleanup(agent):
    if agent:
        try:
            project_client.agents.delete_agent(agent.id)
            print(f"🗑️ Deleted health advisor agent: {agent.name}")
        except Exception as e:
            print(f"❌ Error cleaning up agent: {e}")
    else:
        print("No agent to clean up.")

cleanup(health_advisor)

# Congratulations! 🏆
You've successfully built a **Fun & Fit Health Advisor** that can:
1. **Respond** to basic health and fitness questions.
2. **Use disclaimers** to encourage safe, professional consultation.
3. **Provide** general diet and wellness information.
4. **Use** the synergy of **Azure AI Foundry** modules to power the conversation.

## Next Steps
- Explore adding more advanced tools (like **FileSearchTool** or **CodeInterpreterTool**) to provide more specialized info.
- Evaluate your AI's performance with **azure-ai-evaluation**!
- Add **OpenTelemetry** or Azure Monitor for deeper insights.
- Incorporate **function calling** if you want to handle things like advanced calculation or direct data analysis.

Happy (healthy) coding! 💪